# CS 20 : TensorFlow for Deep Learning Research
## Lecture 04 : Eager execution
### Custon training walkthrough
Categorizing Iris flowers by species by using Tensorflow's eager execution.

This guide uses these high-level TensorFlow concepts:

* Enable an [eager execution](https://www.tensorflow.org/guide/eager?hl=ko) development environment,
* Import data with the [Datasets API](https://www.tensorflow.org/guide/datasets?hl=ko)
* Build models and layers with TensorFlow's [Keras API](https://keras.io/getting-started/sequential-model-guide/)  
  
  
* Reference
    + https://www.tensorflow.org/tutorials/eager/custom_training_walkthrough?hl=ko

### Setup

In [1]:
from __future__ import absolute_import, division, print_function
import os, sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

tf.enable_eager_execution()

print(tf.__version__)

1.12.0


### Create a model

In [2]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, activation = tf.nn.relu, input_shape = (4,)))
model.add(tf.keras.layers.Dense(10, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(3))

In [3]:
def loss_fn(model, features, label):
    score = model(features)
    return tf.losses.sparse_softmax_cross_entropy(labels = label, logits = score)

In [4]:
print(model.trainable_variables)

[<tf.Variable 'dense/kernel:0' shape=(4, 10) dtype=float32, numpy=
array([[-0.30803984, -0.01893753,  0.61146855, -0.47435147,  0.09735668,
        -0.40768373, -0.17765707, -0.5322173 ,  0.4472704 , -0.37332752],
       [-0.49528438,  0.40341616,  0.55749846, -0.16490567,  0.38638103,
        -0.43290883, -0.47683632,  0.6377454 ,  0.2413348 , -0.45055968],
       [ 0.09948409,  0.32634228, -0.5646049 ,  0.34635472,  0.11541736,
         0.64992034, -0.21107197, -0.61615974, -0.6445855 , -0.41054142],
       [-0.51387227, -0.1673187 , -0.16908056, -0.25134146,  0.0717966 ,
         0.2677107 ,  0.2694602 ,  0.35460413, -0.47340906, -0.46411705]],
      dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>, <tf.Variable 'dense_1/kernel:0' shape=(10, 10) dtype=float32, numpy=
array([[-0.50144494, -0.47140843, -0.12475327,  0.4600966 ,  0.26963925,
        -0.07112435, -0.37263134,  0.36020374,  0.400

### Import and parse the training dataset

In [5]:
os.listdir('../data/lecture04/')

['iris_training.csv', 'iris_test.csv']

In [6]:
# define parsing function
def parse_single_example(record):
    decoded = tf.decode_csv(record, [[.0],[.0],[.0],[.0],[]])
    features = decoded[:4]
    label = tf.cast(x = decoded[4], dtype = tf.int32)
    return features, label

In [7]:
epochs = 10
batch_size = 8
learning_rate = .03
tr_dataset = tf.data.TextLineDataset(filenames = '../data/lecture04/iris_training.csv')
tr_dataset = tr_dataset.map(parse_single_example)
tr_dataset = tr_dataset.shuffle(200).batch(batch_size = batch_size)
opt = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
global_step = tf.Variable(0.)

### Train the model

In [8]:
for epoch in range(epochs):
    avg_loss = 0
    tr_step = 0
    
    for mb_x, mb_y in tr_dataset:
        with tf.GradientTape() as tape:
            tr_loss = loss_fn(model, mb_x, mb_y)
        grads = tape.gradient(tr_loss, model.variables)
        opt.apply_gradients(zip(grads, model.variables), global_step = global_step)
        
        avg_loss += tr_loss
        tr_step += 1
    else:
        avg_loss /= tr_step
    
    print('epoch : {:3}, ce_loss : {:.3f}'.format(epoch + 1, avg_loss))

epoch :   1, ce_loss : 1.011
epoch :   2, ce_loss : 0.726
epoch :   3, ce_loss : 0.580
epoch :   4, ce_loss : 0.490
epoch :   5, ce_loss : 0.442
epoch :   6, ce_loss : 0.408
epoch :   7, ce_loss : 0.378
epoch :   8, ce_loss : 0.335
epoch :   9, ce_loss : 0.304
epoch :  10, ce_loss : 0.297


### Evaluate the model on the test dataset

In [9]:
tst_dataset = tf.data.TextLineDataset(filenames = '../data/lecture04/iris_test.csv')
tst_dataset = tst_dataset.map(parse_single_example)
tst_dataset = tst_dataset.batch(batch_size = 30)

In [10]:
tst_x, tst_y = next(iter(tst_dataset))
tst_yhat = tf.argmax(model(tst_x), axis = -1, output_type = tf.int32) 

In [11]:
print('test accuracy : {:.2%}'.format(np.mean(tf.equal(tst_y, tst_yhat))))

test accuracy : 80.00%
